# Energy Load Forecast with Visualization in Interactive Tool

Some explanation

In [54]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact_manual

# Set plotting backend to plotly for interactive plots and plotly-static for static plots
plotting_backend = "plotly"

In [55]:
# generate synthetic data
df = pd.DataFrame()
#hourly dates between 2018-01-01 and 2018-03-31
df['ds'] = pd.date_range(start='2018-01-01', end='2018-12-31', freq='H')
df['y'] = np.sin(df.ds.dt.hour * 7) + np.random.randn(len(df)) * 0.2
df['temp_actual'] = np.random.randn(len(df)) * 5 + 60
df['temp_forecast'] = df.temp_actual + np.random.randn(len(df)) * 5
df.head()

,ds,y,temp_actual,temp_forecast
0,2018-01-01 00:00:00,-0.010073,62.867867,63.635128
1,2018-01-01 01:00:00,0.380021,66.207776,67.259658
2,2018-01-01 02:00:00,0.993040,54.926771,44.949003
3,2018-01-01 03:00:00,0.847828,52.110620,54.496595
4,2018-01-01 04:00:00,0.223327,55.675443,73.202495


## 1. Data Preparation
To better capture different seasons of the year and days of the week, we add conditional seasonaility. For every season of the year (summer, winter, fall and spring) and weekdays/weekend a separate seasonality is modelled. 

In [56]:
# Conditional Seasonality: 4 Seasons for yearly and weekly seasonality
df["summer"] = 0
df.loc[df["ds"].dt.month.isin([6, 7, 8]), "summer"] = 1
df["winter"] = 0
df.loc[df["ds"].dt.month.isin([12, 1, 2]), "winter"] = 1
df["fall"] = 0
df.loc[df["ds"].dt.month.isin([9, 10, 11]), "fall"] = 1
df["spring"] = 0
df.loc[df["ds"].dt.month.isin([3, 4, 5]), "spring"] = 1

# Conditional Seasonality: 4 Seasons, Weekday/Weekend distinction for each season, for daily seasonality
df["summer_weekday"] = 0
df.loc[(df["ds"].dt.month.isin([6, 7, 8])) & (df['ds'].dt.dayofweek.isin([0,1,2,3,4])), "summer_weekday"] = 1
df["summer_weekend"] = 0
df.loc[(df["ds"].dt.month.isin([6, 7, 8])) & (df['ds'].dt.dayofweek.isin([5,6])), "summer_weekend"] = 1

df["winter_weekday"] = 0
df.loc[(df["ds"].dt.month.isin([12, 1, 2])) & (df['ds'].dt.dayofweek.isin([0,1,2,3,4])), "winter_weekday"] = 1
df["winter_weekend"] = 0
df.loc[(df["ds"].dt.month.isin([12, 1, 2])) & (df['ds'].dt.dayofweek.isin([5,6])), "winter_weekend"] = 1

df["spring_weekday"] = 0
df.loc[(df["ds"].dt.month.isin([3, 4, 5])) & (df['ds'].dt.dayofweek.isin([0,1,2,3,4])), "spring_weekday"] = 1
df["spring_weekend"] = 0
df.loc[(df["ds"].dt.month.isin([3, 4, 5])) & (df['ds'].dt.dayofweek.isin([5,6])), "spring_weekend"] = 1

df["fall_weekday"] = 0
df.loc[(df["ds"].dt.month.isin([9, 10, 11])) & (df['ds'].dt.dayofweek.isin([0,1,2,3,4])), "fall_weekday"] = 1
df["fall_weekend"] = 0
df.loc[(df["ds"].dt.month.isin([9, 10, 11])) & (df['ds'].dt.dayofweek.isin([5,6])), "fall_weekend"] = 1

df.head()

,ds,y,temp_actual,temp_forecast,summer,winter,fall,spring,summer_weekday,summer_weekend,winter_weekday,winter_weekend,spring_weekday,spring_weekend,fall_weekday,fall_weekend
0,2018-01-01 00:00:00,-0.010073,62.867867,63.635128,0,1,0,0,0,0,1,0,0,0,0,0
1,2018-01-01 01:00:00,0.380021,66.207776,67.259658,0,1,0,0,0,0,1,0,0,0,0,0
2,2018-01-01 02:00:00,0.993040,54.926771,44.949003,0,1,0,0,0,0,1,0,0,0,0,0
3,2018-01-01 03:00:00,0.847828,52.110620,54.496595,0,1,0,0,0,0,1,0,0,0,0,0
4,2018-01-01 04:00:00,0.223327,55.675443,73.202495,0,1,0,0,0,0,1,0,0,0,0,0


## 2. Model definition
The best parameters for the model can be obtained with hyperparameter tuning. By setting the quantiles, a prediction interval is modelled instead of a point forecast.

In [57]:
# Model and prediction
quantiles = [0.015, 0.985]

params = {
    'n_lags': 7*24, 
    'n_forecasts': 24,
    'n_changepoints': 20, 
    'learning_rate': 0.00213515586423339, 
    'ar_layers': [64, 32, 32, 64],
    'yearly_seasonality': 10,
    'weekly_seasonality': False, 
    'daily_seasonality': False, 
    'epochs': 5, 
    'batch_size': 1024, 
    'quantiles': quantiles,
    }


m = NeuralProphet(**params)
m.set_plotting_backend(plotting_backend)
set_log_level("ERROR")

### 2.1 Lagged and future regressors
With additional information, our model can get better. We use the last 10 days of the actual temperature as lagged regressor and the temperature forecast as the future regressor.

In [58]:
m.add_lagged_regressor(names='temp_actual', n_lags=10*24)
m.add_future_regressor(name='temp_forecast')

### 2.2 Conditional seasonalities

In [59]:
m.add_seasonality(name="summer_weekly", period=7, fourier_order=14, condition_name="summer")
m.add_seasonality(name="winter_weekly", period=7, fourier_order=14, condition_name="winter")
m.add_seasonality(name="spring_weekly", period=7, fourier_order=14, condition_name="spring")
m.add_seasonality(name="fall_weekly", period=7, fourier_order=14, condition_name="fall")

m.add_seasonality(name="summer_weekday", period=1, fourier_order=6, condition_name="summer_weekday")
m.add_seasonality(name="winter_weekday", period=1, fourier_order=6, condition_name="winter_weekday")
m.add_seasonality(name="spring_weekday", period=1, fourier_order=6, condition_name="spring_weekday")
m.add_seasonality(name="fall_weekday", period=1, fourier_order=6, condition_name="fall_weekday")

m.add_seasonality(name="summer_weekend", period=1, fourier_order=6, condition_name="summer_weekend")
m.add_seasonality(name="winter_weekend", period=1, fourier_order=6, condition_name="winter_weekend")
m.add_seasonality(name="spring_weekend", period=1, fourier_order=6, condition_name="spring_weekend")
m.add_seasonality(name="fall_weekend", period=1, fourier_order=6, condition_name="fall_weekend")

## 3. Model training
To test our model later, data is split into training and test data. The test data is the last 5% of the data. The model is trained on the training data.

In [60]:
# Split Train and Test Data 
df_train, df_test = m.split_df(df, valid_p=0.05, local_split=True)
print(f'Train shape: {df_train.shape}')
print(f'Test shape: {df_test.shape}')

Train shape: (8288, 16)
Test shape: (617, 16)


In [61]:
# Fit model
metrics_fit = m.fit(df_train, freq="H", metrics=True)

Training: 0it [00:00, ?it/s]

In [62]:
# Predict
forecast = m.predict(df)

Predicting: 8it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/leoniewagner/Documents/Uni/Stanford/dev_code/neural_prophet/neuralprophet/data/process.py:127: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


WARNING - (py.warnings._showwarnmsg) - /Users/leoniewagner/Documents/Uni/Stanford/dev_code/neural_prophet/neuralprophet/data/process.py:127: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


WARNING - (py.warnings._showwarnmsg) - /Users/leoniewagner/Documents/Uni/Stanford/dev_code/neural_prophet/neuralprophet/data/process.py:127: PerformanceWarnin

In [63]:
# Test
metrics_test = m.test(df_test)

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Loss_test           0.38366252183914185
         MAE_val            1.7904349565505981
        RMSE_val             2.268336296081543
      RegLoss_test                  0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [64]:
# extract season columns and convert to numeric
season_cols = [col for col in forecast.columns if 'season' in col or 'trend' in col]
forecast[season_cols] = forecast[season_cols].apply(pd.to_numeric)

## 4. Visualization

### 4.1 Model plots
NeuralProphet includes plot functions to explore the forecasted data. For an overview over the plot functions see the [Visualizing tutorial](https://neuralprophet.com/how-to-guides/feature-guides/plotly.html).

In [80]:
m.highlight_nth_step_ahead_of_each_forecast(1).plot(forecast.iloc[-10*24:])

In [84]:
m.plot_parameters(components=['trend','trend_rate_change', 'lagged_regressors'])

### 4.2 Interactive tool
Let's explore the data more interactively. By choosing a date, the forecast for the next 24 hours is shown, which represents industry standards. 

In [47]:
# bring into MISO format: from a given date at 2pm, the forecast is for the next day, 24 hours, starting at midnight
def get_day_ahead_format(date, column_name):

    values = pd.DataFrame(columns=['ds', column_name])
    for i in range(0,24):
        if '%' in column_name:
            step_name = f'yhat{i+1} {column_name}' # eg yhat10 1.5%	
        else:
            step_name = f'{column_name}{i+1}'
        ds = date + pd.Timedelta(hours=i+1)
        
        # throw an error if the date is not in the forecast
        if ds not in forecast['ds'].values:
            raise ValueError(f'The date {ds} is not in the forecast. Please choose a date between {forecast["ds"].min()} and {forecast["ds"].max()}')

        step_value = forecast[forecast['ds']==ds][step_name].values[0]
        # concatenate the values
        values = pd.concat([values, pd.DataFrame({'ds':ds, column_name:step_value}, index=[0])], ignore_index=True)

    return values

In [52]:
@interact_manual
def show_forecast_uncertainty(date = widgets.DatePicker()):
    start = pd.to_datetime(date)
    end = start + pd.DateOffset(days=1)

    # get forecast
    np_yhats = get_day_ahead_format(start, 'yhat')
    np_yhats_lower = get_day_ahead_format(start, '1.5%')
    np_yhats_upper = get_day_ahead_format(start, '98.5%')
    np_ar = get_day_ahead_format(start, 'ar')

    # get actual, temp and temp forecast
    forecast_window = forecast[(forecast['ds'] >= start) & (forecast['ds'] < end)]
    df_window = df[(df['ds'] >= start) & (df['ds'] < end)]

    # get components
    seasonalities_to_plot = []
    for season in season_cols:
        if np.abs(forecast_window[season].sum()) > 0:
            seasonalities_to_plot.append(season)

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=('Actuals and Forecast', 'Temperature', 'Forecast Components'))

    # plot LBA actuals as dots not as a line
    fig.add_trace(go.Scatter(x=forecast_window['ds'], y=forecast_window['y'], name='Actuals', mode='markers', marker=dict(symbol='x')), row=1, col=1)

    # plot forecast and uncertainty
    fig.add_trace(go.Scatter(x=np_yhats_lower['ds'], y=np_yhats_lower['1.5%'], fill=None, mode='lines', line_color='#A6B168', name='1.5% quantile'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np_yhats_upper['ds'], y=np_yhats_upper['98.5%'], fill='tonexty', mode='lines', line_color='#A6B168', name='98.5% quantile'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np_yhats['ds'], y=np_yhats['yhat'], name='Forecast', line_color='#7A863B'), row=1, col=1)

    # plot temperature
    fig.add_trace(go.Scatter(x=df_window['ds'], y=df_window['temp_actual'], mode='lines', name='Temperature actual'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df_window['ds'], y=df_window['temp_forecast'], mode='lines', name='Temperature forecast'), row=2, col=1)

    # plot different seasons of seasons_to_plot
    for season in seasonalities_to_plot:
        fig.add_trace(go.Scatter(x=forecast_window['ds'], y=forecast_window[season], name=season), row=3, col=1)
    fig.add_trace(go.Scatter(x=np_ar['ds'], y=np_ar['ar'], name='Autoregression'), row=3, col=1)

    # Updating layout for the first figure (df_LBA)
    fig.update_layout(
        title=f'Forecast on {date}',
        yaxis=dict(title='Load (MW)'), 
        #width=1000,
        height=800
    )
    fig.show()




interactive(children=(DatePicker(value=None, description='date', step=1), Button(description='Run Interact', s…